In [8]:
import numpy as np
import types
import cv2 as cv
# cv.imshow() -> to display image

In [9]:
#function to convert any type of data into binary
def messageToBinary(message):
    if(type(message) == str):
        temp = ''
        for i in message:
            temp = temp + format(ord(i), "08b")
        return temp
    elif(type(message) == bytes or type(message) == np.ndarray):
        temp = []
        for i in message:
            temp.append(format(i, "08b"))
        return temp
    elif(type(message) == int or type(message) == np.uint8):
        return format(message, "08b")
    else:
        raise TypeError('Input Type not supported')


In [10]:
#function to hide the secret message into the image
def hideData(message, image):
    #calculate total number of bits in the image
    no_of_bits = image.shape[0]*image.shape[1]*3 #Since each element or pixel consists of 3 bits -> R,G,B

    #calculate maximum number of bytes to be encoded
    maximum_bytes_encoded = no_of_bits // 8
    print('Maximum number of bytes to encode = ', maximum_bytes_encoded)

    if(len(message) > maximum_bytes_encoded):
        raise ValueError('Error : insufficient image size, warning : data loss')
    
    #set delimiter for the message
    message = message + '***'

    data_index = 0
    no_of_pixels = image.shape[0]*image.shape[1]

    #convert secret message to binary
    binary_secret_message = messageToBinary(message)

    #find length of the data that needs to be hidden
    data_length = len(binary_secret_message)

    for values in image:
        for pixel in values:
            #convert R, G, B to binary format
            r, g, b = messageToBinary(pixel)

            #modify the least significant bit (LSB) only if there is still data to store
            if data_index<data_length:
                #hide each bit of secret message into R, G, B's LSB
                pixel[0] = int(r[:-1] + binary_secret_message[data_index], 2)
                data_index = data_index + 1
                if data_index>=data_length:
                    break
                pixel[1] = int(g[:-1] + binary_secret_message[data_index], 2)
                data_index = data_index + 1
                if data_index>=data_length:
                    break
                pixel[2] = int(b[:-1] + binary_secret_message[data_index], 2)
                data_index = data_index + 1
            else:
                break
    
    return image             


In [11]:
#function to decode the message from the stego image
def decodeMessage(image):
    print('Decoding started')
    binary_data = ''
    for values in image:
        for pixel in values:
            r, g, b = messageToBinary(pixel)
            binary_data = binary_data + r[-1] #extracting data from the LSB of red pixel
            binary_data = binary_data + g[-1] #extracting data from the LSB of green pixel
            binary_data = binary_data + b[-1] #extracting data from the LSB of blue pixel

    print("Length of binary data extracted from stego image = ", len(binary_data))
    #split 8bits into one to convert to byte array
    byte_array = []
    for i in range(0, len(binary_data), 8):
        temp = binary_data[i : i+8]
        byte_array.append(temp)

    #convert bits to characters
    decoded_data = ''
    for item in byte_array:
        decoded_data = decoded_data + chr(int(item,2))
        #if we reached the end and encounter the delimiter
        if decoded_data[-3:] == "***":
            break

    #remove the delimiter to get the original message back
    return decoded_data[:-3]

In [12]:
#encode message into image
def encodeMessage():

    #Read the input image from user
    image_name = input("Enter image name (with extension)")
    image = cv.imread(image_name)

    print("The shape of the image is : ", image.shape)
    #print("The original image is as shown below")
    #Resize image
    #resized_image = cv2.resize(image, (500,500))
    #cv2_imshow(resized_image) #display image

    message = input('Enter the message to be encrypted')
    if len(message) == 0:
        raise ValueError('The message is empty')
    
    encoded_image = hideData(message, image)

    

    filename = input('Enter the name of the new encoded file name to be saved (with extension)')
    cv.imwrite(filename, encoded_image)

In [13]:
def decode_text():
    #read the image that contains the hidden message
    image_name = input("Enter the name of the encoded image (with extension)")
    image = cv.imread(image_name)

    #print("The Stego Image is as shown below")
    #resized_image = cv2.resize(image, (500, 500))
    #cv2_imshow(resized_image)

    text = decodeMessage(image)
    return text

In [14]:
def LSB_Steganography():
    x = input('LSB Image Steganography \n 1.Encode the message \n 2.Decode the message \n Enter your choice : ')
    choice = int(x)
    if choice == 1:
        print('Encoding...')
        encodeMessage()
    elif choice == 2:
        print('Decoding...')
        text = decode_text()
        print("Decoded Text = ", text)
    else:
        raise Exception('Please enter correct input choice')
    
LSB_Steganography() # to encode message

Encoding...
The shape of the image is :  (909, 840, 3)
Maximum number of bytes to encode =  286335


In [15]:
LSB_Steganography() # to decoed message

Decoding...
Decoding started
Length of binary data extracted from stego image =  2290680
Decoded Text =  Steganography is a great art!
